In [1]:
import numpy as np
import pandas as pd
from keras.layers import Conv2D, BatchNormalization, Activation, Input, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from keras import layers
from keras.models import Model
from keras import backend as K
import inception_like_model
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):
    """Utility function to apply conv + BN.
    # Arguments
        x: input tensor.
        filters: filters in `Conv2D`.
        num_row: height of the convolution kernel.
        num_col: width of the convolution kernel.
        padding: padding mode in `Conv2D`.
        strides: strides in `Conv2D`.
        name: name of the ops; will become `name + '_conv'`
            for the convolution and `name + '_bn'` for the
            batch norm layer.
    # Returns
        Output tensor after applying `Conv2D` and `BatchNormalization`.
    """
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x

In [3]:
#
if K.image_data_format() == 'channels_first':
    channel_axis = 1
else:
    channel_axis = 3

# Model architecture
img_input = Input(shape=(200, 200, 2))

x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
x = conv2d_bn(x, 32, 3, 3, padding='valid')
x = conv2d_bn(x, 64, 3, 3)
x = MaxPooling2D((3, 3), strides=(2, 2))(x)

x = conv2d_bn(x, 80, 1, 1, padding='valid')
x = conv2d_bn(x, 192, 3, 3, padding='valid')
x = MaxPooling2D((3, 3), strides=(2, 2))(x)


# mixed 0, 1, 2: 35 x 35 x 256
branch1x1 = conv2d_bn(x, 64, 1, 1)

branch5x5 = conv2d_bn(x, 48, 1, 1)
branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

branch3x3dbl = conv2d_bn(x, 64, 1, 1)
branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
x = layers.concatenate(
    [branch1x1, branch5x5, branch3x3dbl, branch_pool],
    axis=channel_axis)

x = GlobalAveragePooling2D()(x)

# Make model
model = Model(inputs=img_input, outputs=x)

In [4]:
im = np.random.rand(1, 200, 200, 2)

In [5]:
p = model.predict(im)
p

array([[  3.89822349e-02,   0.00000000e+00,   6.75054849e-04,
          5.51655218e-02,   0.00000000e+00,   0.00000000e+00,
          9.54164416e-02,   1.36454357e-03,   5.36442213e-02,
          2.75994521e-02,   0.00000000e+00,   2.19060592e-02,
          5.24031669e-02,   9.81157348e-02,   0.00000000e+00,
          9.38619301e-03,   8.26623105e-03,   0.00000000e+00,
          0.00000000e+00,   7.47322151e-03,   4.16849693e-03,
          0.00000000e+00,   6.60816655e-02,   5.06746722e-03,
          1.91879403e-02,   9.62144211e-02,   1.08637540e-02,
          0.00000000e+00,   1.75407212e-02,   0.00000000e+00,
          1.55635914e-02,   5.82932398e-06,   0.00000000e+00,
          4.63003758e-03,   1.35025922e-02,   5.39142489e-02,
          0.00000000e+00,   5.38773857e-05,   0.00000000e+00,
          0.00000000e+00,   1.65209230e-02,   3.90494317e-02,
          0.00000000e+00,   0.00000000e+00,   5.96953109e-02,
          0.00000000e+00,   0.00000000e+00,   3.94430972e-04,
        

In [6]:
p.shape

(1, 256)